In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/10 13:07:36 WARN Utils: Your hostname, DulanComputer resolves to a loopback address: 127.0.1.1; using 172.30.15.25 instead (on interface eth0)
23/09/10 13:07:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/10 13:07:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
postcode = spark.read.parquet("../../../data/insights/pre_insights/postcode.parquet/")

In [4]:
postcode.show(truncate = False)

+------------+-----------------+-----------+-------------------------------------+--------------------+----------+-----------------------+----------------------------+------------------+-----------------------+---------------------------+-----------------------+------------------+
|merchant_abn|consumer_postcode|consumer_id|merchant_name                        |estimated_population|median_age|median_mortgage_monthly|total_weekly_personal_income|median_weekly_rent|total_weekly_fam_income|avg_num_persons_per_bedroom|total_hhd_income_weekly|avg_household_size|
+------------+-----------------+-----------+-------------------------------------+--------------------+----------+-----------------------+----------------------------+------------------+-----------------------+---------------------------+-----------------------+------------------+
|16570599421 |3332             |1343547    |Non Magna Nam PC                     |8074.0              |40.0      |1733.0                 |780.0           

In [5]:
postcode_agg = postcode.groupBy("merchant_abn").agg\
                (f.first("merchant_name").alias("name"),
                 f.count("consumer_postcode").alias("number_of_postcodes"),
                 f.mode("consumer_postcode").alias("most_popular_postcode"),
                 f.avg("total_weekly_personal_income").alias("avg_total_weekly_personal_income"),
                 f.avg("total_weekly_fam_income").alias("avg_total_weekly_fam_income"),
                 f.avg("median_age").alias("avg_median_age"))

In [15]:
postcode_agg.orderBy(f.col("number_of_postcodes").desc()).show(5,truncate = False)

+------------+---------------------------+-------------------+--------------------------------+---------------------------+------------------+---------------------------------+
|merchant_abn|name                       |number_of_postcodes|avg_total_weekly_personal_income|avg_total_weekly_fam_income|avg_median_age    |avg_num_of_consumers_per_postcode|
+------------+---------------------------+-------------------+--------------------------------+---------------------------+------------------+---------------------------------+
|24852446429 |Erat Vitae LLP             |241547             |789.8770611930597               |1977.015175100498          |43.09343523206664 |91.52974611595302                |
|86578477987 |Leo In Consulting          |227812             |790.1220326409496               |1978.2443615788457         |43.12187242111917 |86.32512315270937                |
|64203420245 |Pede Nonummy Corp.         |217620             |790.7058151824281               |1979.7334918665563  

Checking the proportion of the most popular postcode of the merchant whech served the highest number of consumers in different postcode, i.e, Erat Vitae LLP

In [7]:
EVLLP = postcode.filter(postcode.merchant_abn == 24852446429)
mode_pc_EVLLP = EVLLP.filter(EVLLP.consumer_postcode == 6470).count()
total_pc_EVLLP = 241547
proportion = mode_pc_EVLLP/total_pc_EVLLP
print(proportion)

0.000993595449332842


Using the most popular postcode seems infeasable as the proportion of the most popular postcode is very small.

In [8]:
postcode_agg = postcode_agg.drop("most_popular_postcode")

In [9]:
consumer_id_count_per_postcode = postcode.groupBy("merchant_abn","consumer_postcode").agg(f.count("consumer_id").alias("number_of_consumers"))
avg_num_of_consumers_per_postcode = consumer_id_count_per_postcode.groupBy("merchant_abn").agg(f.avg("number_of_consumers").alias("avg_num_of_consumers_per_postcode"))

In [10]:
postcode_agg = postcode_agg.join(avg_num_of_consumers_per_postcode, on = "merchant_abn", how = "leftouter")

In [16]:
postcode_agg.orderBy(f.col("avg_num_of_consumers_per_postcode").desc()).show(5,truncate = False)

+------------+---------------------------+-------------------+--------------------------------+---------------------------+------------------+---------------------------------+
|merchant_abn|name                       |number_of_postcodes|avg_total_weekly_personal_income|avg_total_weekly_fam_income|avg_median_age    |avg_num_of_consumers_per_postcode|
+------------+---------------------------+-------------------+--------------------------------+---------------------------+------------------+---------------------------------+
|24852446429 |Erat Vitae LLP             |241547             |789.8770611930597               |1977.015175100498          |43.09343523206664 |91.52974611595302                |
|86578477987 |Leo In Consulting          |227812             |790.1220326409496               |1978.2443615788457         |43.12187242111917 |86.32512315270937                |
|64203420245 |Pede Nonummy Corp.         |217620             |790.7058151824281               |1979.7334918665563  

In [12]:
postcode.select("median_age").describe()

summary,median_age
count,11372745
mean,43.107443101907236
stddev,6.327153848420028
min,17.5
max,63.0


In [13]:
postcode_agg.select("avg_median_age").describe()

summary,avg_median_age
count,4026
mean,43.11234880875574
stddev,0.9910923398149496
min,26.0
max,55.0


In [14]:
postcode_agg.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/postcode_agg.parquet")

### Insights into postcodes
- Most consumers are aged between 30-50 years old.
